**Содержание**<a id='toc0_'></a>    
- [Моделирование языка](#toc1_)    
- [Подходы к моделированию языка](#toc2_)    
      - [Авторегрессионные языковые модели](#toc2_1_1_1_)    
      - [Модели со свободным порядком факторизации](#toc2_1_1_2_)    
  - [Классическая N-граммная модель](#toc2_2_)    
  - [Модели SkipGram и CBOW](#toc2_3_)    
  - [Авторегрессионные модели](#toc2_4_)    
  - [Двунаправленные языковые модели. ELMo](#toc2_5_)    
  - [BERT](#toc2_6_)    
  - [Модель XLNet](#toc2_7_)    
  - [Актуальные архитектурные решения в языковых моделях](#toc2_8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Моделирование языка](#toc0_)

\*) наверно все-таки языковая модель?

На **вход** поступает токенизированный текст. 

$$[t_1, t_2, ..., t_l] \sim P(t_1, t_2, ..., t_l)$$

Считаем входящий текст реализацией случайной величины (семпл). 

**Выход**: оценка плотности вероятности $Q(t_1, t_2, ..., t_l) = ?$


Мы строим модель, которая должна уметь, для заданного фрагмента текста, оценивать вероятность встретить такой фрагмент в настоящем тексте (генеральной совокупности), насколько данный фрагмент похож на настоящий текст, или — насколько он правдоподобен. 

В статистике это - задача оценки плотности распределения.

В контексте машинного обучения это является одной из возможных постановок задач обучения без учителя. 

Если у нас есть вычислимая оценка плотности распределения, то мы можем сэмплировать (делать выборки) из этого распределения — то есть, в нашем случае, мы можем генерировать тексты с помощью языковых моделей. 

**Зачем нам это всё надо?**

- Исправление ошибок (proofreading) [1]: 
  - оцениваем правдоподобие набираемого текста с помощью, находим слова, более всего снижающие правдоподобие текста, и подсвечиваем их как возможное место, где есть ошибка. 
- Предложение вариантов при генерации текстов:
  - распознавание речи (let it be / let eat bee)
  - машинный перевод
- Использование неразмеченных данных для обогащения моделей машинного обучения
  - **перенос знаний**, перенос навыков (transfere learning): в обучении с учителем размеченные данные обычно сравнительно небольшие. Модель, обученная без учителя, будет отражать гораздо большее число харакетристик данных, чем только одни метки классов
  - предобучение (pretraining), дообучение (fine-tuning), извлечение признаков (feature extration): 
    - Мы можем обучить большую модель на большом корпусе — предпочтительно, без учителя (если можем, конечно). 
    - Потом — использовать небольшой корпус, предназначенный для одной конкретной задачи, чтобы "донастроить" полученную большую модель именно под эту задачу. 

**Языковые модели** — это основа переноса навыков в обработке текстов. Это как нейросети, обученные на ImageNet, если проводить аналогию с областью компьютерного зрения.

[1] https://habr.com/ru/post/108831/

# <a id='toc2_'></a>[Подходы к моделированию языка](#toc0_)

Даны тексты $[t_1, t_2, ..., t_l] \sim P(t_1, t_2, ..., t_l)$, $P$ - неизвестно (некоторое многомерное распределение)

Правило цепочки (chain rule):

$$P(a, b, c) = P(a|b, c) P(b|c) P(c) = P(b|a, c) P(c|a) P(a)$$

- совместное распределение нескольких случайных величин можно представить в виде произведения условных распределений **в различном порядке**.

Порядок, в котором записывается условные распределение, называется порядком факторизации (англоязычный термин factorization order). В зависимости от того, как мы раскручиваем эту цепочку, мы можем получить разные постановки задачи моделирования языка.

#### <a id='toc2_1_1_1_'></a>[Авторегрессионные языковые модели](#toc0_)

Классическая и одна из самых популярных постановок (**жесткий порядок факторизации**), когда мы предсказываем слово за словом. 

$$P(t_1, t_2, ..., t_l) = P(t_l|t_1,t_2,, ..., t_{l-1})P(t_{l-1}|t_1,, ..., t_{l-2}))...P(t_1)
\\P(t_1, t_2, ..., t_l) = \prod_{i=1}^l P(t_i|t_{1:i-1})$$

- Вероятность следующего слова при наблюдении какого-то количества предыдущих слов. 
- К этой группе относятся популярные модели — такие, как **ELMo, GPT, OpenAI Transformer[1], XLNet**

#### <a id='toc2_1_1_2_'></a>[Модели со свободным порядком факторизации](#toc0_)

Другая постановка подразумевает отсутствие жёстко заданного порядка факторизации. 

Такие модели предсказывают слова в середине фрагмента текста по остальным словам. 

К таким моделям относиться **word2vec, CBOW, BERT, XLNet**.

[1] Попробуйте пообщаться с трансформером: https://talktotransformer.com/ (лучше, на английском)

## <a id='toc2_2_'></a>[Классическая N-граммная модель](#toc0_)

Простейшая авторегрессионная модель (AR-модель).

$$P(t_1, t_2, ..., t_l) = \prod_{i=1}^l P(t_i|t_{1:i-1})$$

Длина истории ограничена $n$: $P(t_i|t_{1:i-1}) = P(t_i|t_{i-n:i-1})$
- не учитываем вероятности слов далее $n$

Это ограничение называется **свойство Марковости n-го порядка**:
- $n=1$ это простейшая марковская цепь

Все вероятности хранятся в модели в явном виде (с помошью хэш-таблицы или префиксного дерева)

Обучение - это подсчет частот: $P(t_i|t_{i-n:i-1}) = \frac{P(t_{i-n:i})} {P(t_{i-n:i-1})}$

Преимущества:
- простота реализации 
- скорость

Недостатки:
- при увеличении N, количество N-грамм соответствующей длины растёт в геометрической прогрессии (на практике N не превышает 3)
- полученные представления сложно закодировать в признаки для дальнейшей работы
- сильное обобщение, сжатие представления
- нет работы с неизвестными словами (нет вектора - нет расстояния)
  - частично решается переходом с токенов на символы, но тут в полный рост встает проблема большого N для N-грамм
- такие модели ничего не знают про синтаксический анализ, про сложную многосвязную структуру текста. 
  - всё, что они видят — это только сколько-то слов слева от текущего. Другими словами они не особо-то выразительны.

## <a id='toc2_3_'></a>[Модели SkipGram и CBOW](#toc0_)

**SkipGram**: предсказание соседних слов в некотором окне при условии наблюдения центрального слова

$$P(t_{k+m}|t_{k}), m = \{-d, -d+1, -d+2, ..., d\}$$

**Continuous Bag-of-words (CBoW)**: предсказывает центральное слово по остальным словам, в рамках окна

$$P(t_k|t_{k-d},..,t_{k-1},t_{k+1},..,t_{k+d} )$$

- Принципиальное отличие обоих моделей от классической N-граммной модели - векторное представление в виде 2 векторов (центральный "w" и контекстный "d")

- Вероятности считаются через $softmax$ от скалярного произведения векторов
  - softmax: отображает вещественный вектор в такой вектор положительных чисел, что сумма координат всегда равна 1
  - neg.sampling: тоже самое, но по небольшой подвыборке словаря
  - есть еще иерархический софтмакс[1]

$$P(t_j|t_i) = \frac {e^{w_j \cdot d_i}}{\sum_{j=1}^|V| e^{w_j \cdot d_i}} = \text{softmax} \approx \text{neg.sampling} = \frac {e^{w_j \cdot d_i^+}}{\sum_{j=1}^M e^{w_j \cdot d_i^-}}, M \ll |V|$$

- Настраиваются модели word2vec (а это модели данного класса) с помощью **градиентного спуска** и в качестве целевой функции выступает **кросс-энтропия**.

- По сути, модель языка решает задачу **классификации**, в которой количество классов равно размеру словаря.

- можно оценить правдоподобие текста:
$$P(t_1, t_2, ..., t_l) \approx \prod_{i=1}^l \prod_{j=1, j\neq i}^l P(t_j|t_i)$$

**Преимущества:**
- простота
- сжатое представление, вектора удобны для использования в других алгоритмах в рамках конвееров
- арифметика смыслов
- работа сбольшими словарями (порядка $10^6$)

**Недостатки:**
- нет работы с неизвестными словами
  - но зато **FastText** умеет
- невозможность учета сложного контекста
  - у нас только 1 вектор для слова, он характеризует только 1 смысл

[1] Hierarchical softmax and negative sampling: short notes worth telling  
[2] https://dumps.wikimedia.org/



## <a id='toc2_4_'></a>[Авторегрессионные модели](#toc0_)

Вот тут (с учетом недостатков word2vec по работе с контекстом) на сцену выходят **авторегрессионные языковые модели на основе нейросетей**. 

Модель:

- каждый токен по-отдельности преобразуется в вектор, получается embedding токена без учёта контекста (правило цепочки)

$$P(t_1, t_2, ..., t_l) = \prod_{i=1}^l P(t_i|t_{1:i-1})$$

- несколько слоев рекуррентных (RNN), свёрточных (CNN) или архитектур с вниманием (Self-attention/Transformer decoder) - происходит учёт контекста, 
  
- на последнем слое - предсказываются вероятности слов. 
  
Чаще всего в последнее время используется честный софтмакс, а не его аппроксимация, так как использование приближений несколько ухудшает результаты. 

Поэтому важно не раздувать словарь слишком сильно — на практике, многие большие языковые модели работают со словарями размера не больше 50 000 уникальных токенов. 

Очевидно, что при этом мы отбрасываем очень большое количество слов. 

Как с этим работать:
- символьные N-граммы
- FastText эмбеддинги (для неизвестных слов вектор строится как наиболее близкий по символьным эмбеддингам)
- промежуточный вариант (**byte pair encoding**) 

**byte pair encoding**

Простой алгоритм кодирования, который позволяет сжать длину последовательностей за счёт расширения словаря (то есть можно выбирать разные соотношения длины последовательностей и размера словаря).

Этот алгоритм работает в два этапа:

- обучение - построение правил замены символов
  - самая часто встречающаяся биграмма заменяется на новый символ, который раньше не использовался
  - находится новая биграмма и всё повторяется
- кодирование - применение построенных правил замены, чтобы сократить длину текста

Это позволяет сильно сократить размер словаря и, при этом, не сильно увеличить длину входных цепочек токенов, в отличие от символьных языковых моделей ("aabc aabaa" -> "Xbc XbX -> "Yc YX")

**Преимущества:**
- сжатое представление, обобщение. В построенных так моделях
  - первые слои выделяют признаки (морфология, части речи и т.п.)
  - средние слои отвечают за согласование слов (синтаксис)
  - последние слои передают смысл (семантические роли, смыслы)
  - То есть всё, практически, как в работе с изображениями — на каждом новом уровне мы абстрагируемся всё больше и больше, выделяем всё более крупные паттерны
- Учет сложного контекста
- Хорошая работа с неизвестными словами (если byte pair encoding)

**Проблемы:**
- вычислительная сложность (хорошая модель до сходимости требует несолько дней вычислений на десятках GPU)
- математические прикольчики (RNN дают затухание/вызрыв градиента)
- длина контекста слева ограничена (особенно если в основе CNN)

Если строить авторегрессионную языковую модель с помощью **свёрточных нейросетей**, сложно учитывать широкий контекст, так как ширину рецептивного поля свёрточных нейросетей можно увеличивать только вместе с количеством параметров (добавляя слои).

Если строить авторегрессионную языковую модель с помощью **рекуррентных нейросетей**, то с ростом длины последовательности острее проявляется проблема затухания градиента и забывания со временем.

## <a id='toc2_5_'></a>[Двунаправленные языковые модели. ELMo](#toc0_)

В русском языке свободный порядок слов. Авторегрессионная модель учитывает контекст только слева. А что, если сделать две авторегрессионных модели, которые будут получить один и тот же текст в противоположном порядке.

$$P_{left}(t_1, t_2, ..., t_l) = \prod_{j=1}^l P(t_j|t_{1:i-1}) \\
P_{right}(t_1, t_2, ..., t_l) = \prod_{j=1}^l P(t_j|t_{i+1:l})$$

- При этом, признаки от обеих моделей можно использовать для лучшего решения прикладных задач. Скрытые состояния со всех шагов и со всех слоев используются как признаки для решения прикладной задачи

Такой подход лежит в основе модели **ELMo**, которая была предложена в 2017-2018 годах.[1] 

[1] Peters M. E. et al. Deep contextualized word representations //arXiv preprint arXiv:1802.05365. – 2018. (https://arxiv.org/abs/1802.05365)  



## <a id='toc2_6_'></a>[BERT](#toc0_)

<img src="./img/bert.png" width="700">

Однако можно пойти ещё дальше и учитывать не только контекст строго слева или строго справа, а вообще весь доступный текст, кроме некоторых слов. 

- Так как модель основана на **трансформере** (далее подробнее), у неё нет особых проблем с моделированием далёких зависимостей — это позволяет моделировать не только одно предложение, а сразу большой непрерывный фрагмент текста. 
  - 12 или 14 слоев
- При обучении модели BERT, берётся фрагмент текста, в нём выбирается сколько-то слов, и они заменяются на фиктивный токен ([MASK]). 
- Модель должна предсказать эти слова, используя все остальные входные данные. Тут учитывается вообще весь текст. 
$$P(t_1^{mask}, t_2^{mask}, ..., t_m^{mask}|t_1^{input}, t_2^{input}, ..., t_n^{input})$$
  
**Преимущества:**
- предсказывается еще и следующее предложение
- учитывается контекст справа
- SOTA, гибкость
  
Данная модель позволяет достичь лучших на настоящее время результатов на ряде задач, при этом не обязательно строить какие-то специальные архитектуры для каждой задачи — можно, не меняя общей архитектуры BERT, несколько до-обучить его на маленьком датасете, и задача уже будет решаться отлично. 
  
**Проблемы:**
- пропущенные слова [MASK] считаются условно-независимыми при наблюдении остальных слов. 
  $$P(t_1^{mask}, t_2^{mask}, ..., t_m^{mask}|t_{1:n}^{input}) = \prod_{i=1}^m P(t_i^{mask}|t_{1:n}^{input})$$
  - Это означает, что их можно предсказывать независимо друг от друга — это не вполне корректно, так как от выбора одного пропущенного слова может зависеть выбор второго. 
- т.к. процесс обучения BERT опирается на внесение шума во входные данные (добавляется фиктивный токен [MASK]) фактическое распределение входных данных не вполне соответствует настоящему. 
  - Это может препятствовать переносу на реальные данные. 

Ну, как бы то ни было, в целом эта модель работает отлично.


## <a id='toc2_7_'></a>[Модель XLNet](#toc0_)

Позже была предложена модель XLNet[1], показывающая ещё лучшее качество, чем BERT: объединение авторегрессионных моделей и моделей, основанных на восстановлении зашумлённых данных — таких как BERT. 

Правило цепочки (chain rule) может быть представлено в виде различных последовательностей:

$$P(a, b, c) = P(a|b, c) P(b|c) P(c) = P(b|a, c) P(c|a) P(a)$$

- первый вариант соответствует цепочке C-B-A, 
- второй вариант соответствует цепочке A-C-B. 
- авторегрессионные модели (ELMp, OpenAI GPT) всегда слева-направо $P(t_1, t_2, ..., t_l) = \prod_{i=1}^l P(t_i|t_{1:i-1})$
- BERT использует сразу все входные данные — и слева, и справа: $P(t_1^{mask}, t_2^{mask}, ..., t_m^{mask}|t_{1:n}^{input}) = \prod_{i=1}^m P(t_i^{mask}|t_{1:n}^{input})$ (но в итоге работает не совсем с тем текстом, который есть, а с зашумленным вариантом). 
 
XLNet: используется обычная авторегрессионная модель для предсказания слова за словом, но для каждого следующего обучающего примера использовать новый порядок факторизации. 
$$P(t_1, t_2, ..., t_l) = \prod_{i=1}^l P(t_{order(i)}|t_{order(1:i-1)}), order \in AllPermutations(1, .., l) $$

XLNet основан на трансформере. Это позволяет очень удобно всё сделать за счёт передачи масок в механизм внимания. Это, на сегодняшний день, самая мощная языковая модель[2].

[1] [Yang Z. et al. Xlnet: Generalized autoregressive pretraining for language understanding //Advances in neural information processing systems. – 2019. – С. 5754-5764.](https://arxiv.org/abs/1906.08237)   
[2] Прогресс можно отслеживать на страницах [GLUE Benchmark](https://gluebenchmark.com/leaderboard) и [SuperGLUE Benchmark](https://super.gluebenchmark.com/leaderboard)

    - уже не в топе
    - в топе некая ансамблевая модель с трансформерами

**Пример**

Допустим, мы выбрали порядок факторизации 3 -> 1 -> 2 для предложения "мама мыла раму". Это означает, что будут формироваться следующие обучающие примеры (в формате "Вход -> Эталонный выход", в скобках после слова стоит его исходная позиция в тексте):

    <start> -> раму(3)
    <start> раму(3) -> мама(1)
    <start> раму(3) мама(1) -> мыла(2)

Так как порядок слов в исходном предложении очень важен для понимания его смысла, необходимо добавлять к эмбеддингу слов эмбеддинг его исходной позиции в тексте. Это называется **позиционным кодированием**

## <a id='toc2_8_'></a>[Актуальные архитектурные решения в языковых моделях](#toc0_)

- модели, использующие сжатие текста и словаря через "byte pair encoding" и, при этом, использующие честный софтмакс (а не его аппроксимацию)

- модель трансформер (заняла доминирующее положение), так как она легче обучается и способен моделировать далёкие зависимости
  - в механизм внимания можно подавать маски и, тем самым, управлять зависимостями между словами
  - лучше обучать модель на длинных входных последовательностях (вносит в модель информацию о связи предложений, о дискурсе)

Семинар в директории `./stepik-dl-nlp`